## 전처리 개요

- 대상 파일: 전처리전_아고다.csv, 전처리전_야놀자.xlsx
- 전처리 항목
    - 특수문자 제거
    - 의미없는 자음, 모음 제거
    - 이모티콘 제거
    - 맞춤법 고치기 (띄어쓰기, 맞춤법등 변경해주는 네이버 맞춤법 검사 api 활용)

## 파일 import 

In [2]:
import pandas as pd

In [6]:
df1 = pd.read_csv('../data/전처리전_아고다.csv')
df2 = pd.read_excel('../data/전처리전_야놀자.xlsx')

In [7]:
df1.head(1)

,score,text,title
0,8.0,ㅇ,글래드라이브 호텔


In [8]:
df2.head(2)

,Unnamed: 0,hotel,star,review,date,length
0,0,노보텔 앰배서더 서울 용산,5,역시 시설도 깨끗하고 좋았어요친절했구요,2020. 12. 02,21
1,1,노보텔 앰배서더 서울 용산,4,되게좋앗어요,2020. 12. 01,6


In [10]:
df2 = df2.iloc[: ,1:]

In [12]:
df2.head(1)

,hotel,star,review,date,length
0,노보텔 앰배서더 서울 용산,5,역시 시설도 깨끗하고 좋았어요친절했구요,2020. 12. 02,21


## 파일 통일

- df1, df2의 column명을 통일하고, concat으로 붙여주는 작업진행

In [13]:
df1.columns = ['score','review','site']

In [15]:
df2 = df2[['star','review','hotel']]
df2.columns = ['score','review','site']

In [17]:
df=pd.concat([df1,df2])

## 특수문자 제거

In [18]:
# 각 전처리 후, 18글자 이하의 리뷰 갯수 비율 확인하는 함수 
def del_percent():
    under_18 = df.loc[df['length'] <= 18,'review'].count()
    all_count = df['review'].count()
    print('18글자 이하 리뷰갯수:', under_18)
    print(under_18 / all_count * 100)

In [26]:
# 리뷰의 length 측정해서 컬럼으로 넣어줌
df['length'] = df['review'].apply(lambda x: len(str(x)))
df['review']=df['review'].astype('str')

### \n (개행문자) 제거

In [27]:
#1.'\n' -> ''
df['review'] = df['review'].apply(lambda x: x.replace('\n',''))
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 1296
54.729729729729726


### 텍스트에 포함되어 있는 특수 문자 제거

In [28]:
import re
# 2. ~, !, ., >
def cleanText(readData):
 
    
    text = re.sub('[-=+,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤]', '', readData)
    return text

df['review'] = df['review'].apply(cleanText)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 1297
54.77195945945946


### 의미없는 자음, 모음만 쓴 것 처리

In [29]:
def cleanText2(readData):
 
    text = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', readData)
    return text
df['review'] = df['review'].apply(cleanText2)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 1297
54.77195945945946


### 이모티콘 제거

In [30]:
def cleanText3(readData):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', readData)
    return text

df['review'] = df['review'].apply(cleanText3)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 1297
54.77195945945946


## 글자수 cut
- 500자 이상 제거 - 16개
- 500자 이상 data 2개 있으며, outlier 이다. 
- 네이버 맞춤법 검사 라이브러리 돌리려면 500자 이하여야 한다.

In [31]:
df[df['length'] > 500].shape

(2, 4)

In [32]:
df = df[df['length'] <= 500]
df.shape

(2366, 4)

## 맞춤법 검사 돌리기

<a href='https://github.com/ssut/py-hanspell'> 라이브러리 다운로드 받기</a>

In [33]:
#맞춤법검사기 
from hanspell import spell_checker

In [17]:
def spell_check(review):
    result = spell_checker.check(review)
    return result.checked

In [18]:
df['review']=df['review'].astype('str')

In [19]:
df['review_spell_check'] = df['review'].apply(spell_check)

In [24]:
df.head()

,score,review,site,length,review_spell_check
0,8.0,,글래드라이브 호텔,0,
1,8.8,글래드 계열 호텔 침구은 언제나 편안하고 룸 청결 상태는 항상 좋습니다,글래드라이브 호텔,39,글래드 계열 호텔 침구는 언제나 편안하고 룸 청결 상태는 항상 좋습니다
2,10.0,오랜전부터 출장 때문에 자주 투숙하고 있습니다 침대도 편안하고 모든게 좋은데 가격은...,글래드라이브 호텔,96,오랜 전부터 출장 때문에 자주 투숙하고 있습니다 침대도 편안하고 모든 게 좋은데 가...
3,6.4,지하철 역이랑 가까워서 좋아요,글래드라이브 호텔,16,지하철역이랑 가까워서 좋아요
4,9.2,넓은 침대와 긴 쇼파 2개로 5명까지 우리는 편하게 잤다,글래드라이브 호텔,31,넓은 침대와 긴 소파 2개로 5명까지 우리는 편하게 잤다


## 파일 내보내기

In [83]:
df.to_excel('data/all.xlsx')